In [1]:
# print('### OS ###')
# !cat /etc/issue.net

# print('### CPU ###')
# !cat /proc/cpuinfo

# print('### MEM ###')
# !cat /proc/meminfo

# print('### DISK ###')
# !df -h

# print('### GPU ###')
# !nvidia-smi

In [1]:
import tensorflow as tf
print(tf.__version__)

import torch
import torchvision
import torchtext
print(torch.__version__)

2.4.1
1.7.1+cu101


In [2]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from gym.envs.registration import register

# 환경 설정. 미끄러지지 않고, 맵을 4 * 4로 한다.
register(
    id='FrozenLake-v3',
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name': '4x4',
            'is_slippery': False}
)

# 환경을 만든다.
env = gym.make("FrozenLake-v3")

# 환경의 상태를 출력한다
env.render() 


SFFF
FHFH
FFFH
HFFG


# add random noise

In [33]:
state = env.reset()
env.render() 

# 상태와 액션의 쌍으로 된 공간을 만들고 0으로 초기화 = Q 함수를 0으로 초기화한다
Q = np.zeros([env.observation_space.n, env.action_space.n]) 

dis = .9  # discount factor
num_episodes = 10000  # 게임 수행하며 학습할 횟수
rList = []  # 게임 결과 저장. (reward의 총합)

for i in range(num_episodes):
  # 새 게임이므로 상태 초기화한다.
  # Q 함수를 초기화하지 않는 것에 주목
  state = env.reset()
  rAll = 0
  done = False

  # 게임 종료될 때까지 (goal을 찾을 때까지) 반복한다.
  while not done:
    # 현재 상태에서 어떤 행동을 할지 결정한다.
    # Q 함수의 값 + 랜덤 노이즈 + decaying이 적용되었다.
    action = np.argmax(Q[state, :] + np.random.randn(1, env.action_space.n) / (i + 1))

    # 결정한 행동을 취한 결과에 대한 상태와 보상 등의 정보를 가져온다.
    new_state, reward, done, _ = env.step(action)

    # 상태와 행동 쌍의 공간에 결과를 기록한다 = Q 함수를 갱신한다.
    # future reward에 dis가 곱해진 것을 확인하자.
    Q[state, action] = reward + dis * np.max(Q[new_state, :])

    # goal 까지 가는 동안의 reward를 모두 더한다. (이 예제의 경우 찾으면 1 아니면 0이된다.)
    rAll += reward
    state = new_state
  
  rList.append(rAll)

print('success rate: ' + str(sum(rList)/num_episodes))
print('Final Q-table values')
print('left down right up')
print(Q)
# plt.bar(range(len(rList)), rList, color='blue')
# plt.show()


SFFF
FHFH
FFFH
HFFG
success rate: 0.9899
Final Q-table values
left down right up
[[0.      0.59049 0.      0.     ]
 [0.      0.      0.      0.     ]
 [0.      0.      0.      0.     ]
 [0.      0.      0.      0.     ]
 [0.      0.6561  0.      0.     ]
 [0.      0.      0.      0.     ]
 [0.      0.      0.      0.     ]
 [0.      0.      0.      0.     ]
 [0.      0.      0.729   0.     ]
 [0.      0.81    0.      0.     ]
 [0.      0.9     0.      0.     ]
 [0.      0.      0.      0.     ]
 [0.      0.      0.      0.     ]
 [0.      0.      0.9     0.     ]
 [0.      0.      1.      0.     ]
 [0.      0.      0.      0.     ]]


# E-greedy


In [6]:
state = env.reset()
env.render() 

# 상태와 액션의 쌍으로 된 공간을 만들고 0으로 초기화 = Q 함수를 0으로 초기화한다
Q = np.zeros([env.observation_space.n, env.action_space.n]) 

dis = .9  # discount factor
num_episodes = 2000  # 게임 수행하며 학습할 횟수
rList = []  # 게임 결과 저장. (reward의 총합)

for i in range(num_episodes):
  # 새 게임이므로 상태 초기화한다.
  # Q 함수를 초기화하지 않는 것에 주목
  state = env.reset()
  rAll = 0
  done = False
  e = 1. / ((i / 100) + 1)
  # 게임 종료될 때까지 (goal을 찾을 때까지) 반복한다.
  while not done:
    # 현재 상태에서 어떤 행동을 할지 결정한다.
    # Q 함수의 값 + 랜덤 노이즈 + decaying이 적용되었다.
    if np.random.rand(1) < e:
      action = env.action_space.sample()
    else:
      action = np.argmax(Q[state, :])

    # 결정한 행동을 취한 결과에 대한 상태와 보상 등의 정보를 가져온다.
    new_state, reward, done, _ = env.step(action)

    # 상태와 행동 쌍의 공간에 결과를 기록한다 = Q 함수를 갱신한다.
    # future reward에 dis가 곱해진 것을 확인하자.
    Q[state, action] = reward + dis * np.max(Q[new_state, :])

    # goal 까지 가는 동안의 reward를 모두 더한다. (이 예제의 경우 찾으면 1 아니면 0이된다.)
    rAll += reward
    state = new_state
  
  rList.append(rAll)

print('success rate: ' + str(sum(rList)/num_episodes))
print('Final Q-table values')
print('left down right up')
print(Q)
# plt.bar(range(len(rList)), rList, color='blue')
# plt.show()


SFFF
FHFH
FFFH
HFFG
success rate: 0.8075
Final Q-table values
left down right up
[[0.531441   0.59049    0.4782969  0.531441  ]
 [0.531441   0.         0.43046721 0.4782969 ]
 [0.4782969  0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.59049    0.6561     0.         0.531441  ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.6561     0.         0.729      0.59049   ]
 [0.6561     0.81       0.81       0.        ]
 [0.729      0.9        0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.81       0.9        0.729     ]
 [0.81       0.9        1.         0.81      ]
 [0.         0.         0.         0.        ]]
